In [1]:
%load_ext jupyternotify
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

<IPython.core.display.Javascript object>

In [2]:
raw_18 = pd.read_csv('../data/PitchData_18.csv', index_col=[0])
raw_19 = pd.read_csv('../data/PitchData_19.csv', index_col=[0])
raw_20 = pd.read_csv('../data/PitchData_20.csv', index_col=[0])
raw_21 = pd.read_csv('../data/PitchData_21.csv', index_col=[0])

In [3]:
raw_all = pd.concat([raw_21, raw_20, raw_19], ignore_index=True)
data = raw_all.copy()

In [4]:
data = data[data['game_type']=='R'].copy()

In [5]:
data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
10786,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,0.69,1.38,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,NaN,NaN,518595.0,NaN,NaN,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,0,5,0,5,Infield shift,Strategic,148.0,0.000,-0.073
10787,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,NaN,NaN,NaN,NaN,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-0.77,0.48,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,0,5,0,5,Infield shift,Strategic,315.0,0.000,-0.027
10788,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,NaN,NaN,NaN,NaN,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-0.65,-0.51,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,0,5,0,5,Infield shift,Standard,328.0,0.000,-0.020
10789,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,NaN,ball,NaN,NaN,NaN,NaN,12.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,B,NaN,NaN,0,0,2021,-0.69,-0.69,2.10,3.89,NaN,NaN,NaN,2,9,Top,NaN,NaN,NaN,NaN,518595.0,NaN,NaN,2.375830,-109.205830,2.277617,-5.902656,19.427562,-38.284747,3.31,1.48,NaN,NaN,NaN,74.5,2017.0,5.9,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,Curveball,5,0,0,5,0,5,0,5,Standard,Standard,330.0,0.000,0.016
10790,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,"Brandon Nimmo grounds out, second baseman Ozzi...",R,L,L,ATL,NYM,X,4.0,ground_ball,1,0,2021,0.63,1.28,-0.31,2.80,NaN,NaN,NaN,1,9,Top,144.83,145.72,NaN,NaN,518595.0,NaN,NaN,-5.868477,-132.500539,-6.486796,8.700586,30.117690,-15.941174,3.49,1.60,9.0,93.3,-18.0,90.9,2281.0,6.3,632254,519293,518595.0,663586.0,645277.0,501303.0,606115.0,592696.0,628338.0,594807.0,54.15,0.100,0.090,0.0,1.0,0.0,0.0,2.0,60,2,4-Seam Fastball,5,0,0,5,0,5,0,5,Standard,Standard,143.0,0.002,-0.189


# 1. Calculate RE24

Currently, each on base column is labeled by the ID of the player that occupies it, and by NAN if no one is on that base. I will turn them into 0 and 1 for calculation.

In [6]:
# replace onbase info with 0 and 1
data.loc[data['on_3b'].notna(), 'on_3b'] = 1
data.loc[data['on_2b'].notna(), 'on_2b'] = 1
data.loc[data['on_1b'].notna(), 'on_1b'] = 1
data[['on_3b', 'on_2b', 'on_1b']] = data[['on_3b', 'on_2b', 'on_1b']].fillna(0)

## 1.1 Calculate Run Scored till End of Inning

In [7]:
# group data by plate appearance
data_pa = data.groupby(['game_date', 'game_pk', 'at_bat_number']).agg({'game_year':max,
                                                                       'home_team':max,
                                                                       'away_team':max,
                                                                       'player_name':max,
                                                                       'batter':max,
                                                                       'pitcher':max,
                                                                        'inning_topbot':max,
                                                                        'inning':max,
                                                                        'outs_when_up':max,
                                                                        'home_score':'last',
                                                                        'away_score':'last',
                                                                        'post_home_score':'first',
                                                                        'post_away_score':'first',
                                                                        'bat_score':'last',
                                                                        'post_bat_score':'first',
                                                                        'on_1b':'last',
                                                                        'on_2b':'last',
                                                                        'on_3b':'last',
                                                                        'events':'first',
                                                                        'delta_home_win_exp':sum,
                                                                        'delta_run_exp':sum}).reset_index().copy()

In [8]:
data_pa['PA_score'] = data_pa['post_bat_score'] - data_pa['bat_score'] # calculate the run scored on that PA

In [9]:
# group data by inning to get the final score of the inning
data_inning = data_pa.groupby(['game_date', 'game_pk', 
                               'home_team', 'away_team', 
                               'inning_topbot', 'inning']).agg({'post_bat_score':max}).reset_index().copy()
data_inning.rename(columns={'post_bat_score':'inning_score'}, inplace=True)

In [10]:
data_inning.head()

,game_date,game_pk,home_team,away_team,inning_topbot,inning,inning_score
0,2019-03-20,566083,OAK,SEA,Bot,1,1
1,2019-03-20,566083,OAK,SEA,Bot,2,2
2,2019-03-20,566083,OAK,SEA,Bot,3,4
3,2019-03-20,566083,OAK,SEA,Bot,4,4
4,2019-03-20,566083,OAK,SEA,Bot,5,4


In [11]:
# add inning score back to the PA data in order to get the run scored till end of inning
data_pa = data_pa.merge(data_inning, on=['game_date', 'game_pk', 'home_team', 
                                         'away_team', 'inning_topbot', 'inning'], how='outer').copy()

In [12]:
# calculate run scored from start of PA till end of inning
data_pa['inning_score_diff'] = data_pa['inning_score'] - data_pa['bat_score']

In [13]:
data_pa.shape

(434372, 27)

In [14]:
data_pa.head()

,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff
0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0
1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0
2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0
3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0
4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1


## 1.2 Drop all bottom of last inning

In [15]:
# get the final inning number of each game
final_inning = data_pa.groupby(['game_date', 'game_pk']).agg({'inning':max}).reset_index().copy()
final_inning.rename(columns={'inning':'final_inning'}, inplace=True)
# merge final_inning with data_pa
data_pa = data_pa.merge(final_inning, on=['game_date', 'game_pk'], how='outer').copy()

In [16]:
# drop bot last innings
data_pa_drop = data_pa[~((data_pa.inning_topbot=='Bot')&(data_pa.inning==data_pa.final_inning))].copy()
print(f"PA dropped: {len(data_pa) - len(data_pa_drop)}")

PA dropped: 12533


## 1.3 Calculate RE24

In [17]:
# combine on base information into one "on_base" column
data_pa_drop.loc[(data_pa_drop['on_1b']==0)&(data_pa_drop['on_2b']==0)&(data_pa_drop['on_3b']==0), 'on_base'] = 0
data_pa_drop.loc[(data_pa_drop['on_1b']==1)&(data_pa_drop['on_2b']==0)&(data_pa_drop['on_3b']==0), 'on_base'] = 1
data_pa_drop.loc[(data_pa_drop['on_1b']==0)&(data_pa_drop['on_2b']==1)&(data_pa_drop['on_3b']==0), 'on_base'] = 2
data_pa_drop.loc[(data_pa_drop['on_1b']==0)&(data_pa_drop['on_2b']==0)&(data_pa_drop['on_3b']==1), 'on_base'] = 3
data_pa_drop.loc[(data_pa_drop['on_1b']==1)&(data_pa_drop['on_2b']==1)&(data_pa_drop['on_3b']==0), 'on_base'] = 12
data_pa_drop.loc[(data_pa_drop['on_1b']==1)&(data_pa_drop['on_2b']==0)&(data_pa_drop['on_3b']==1), 'on_base'] = 13
data_pa_drop.loc[(data_pa_drop['on_1b']==0)&(data_pa_drop['on_2b']==1)&(data_pa_drop['on_3b']==1), 'on_base'] = 23
data_pa_drop.loc[(data_pa_drop['on_1b']==1)&(data_pa_drop['on_2b']==1)&(data_pa_drop['on_3b']==1), 'on_base'] = 123
# calculate RE24
re24 = data_pa_drop.groupby(['on_base', 'outs_when_up']).agg({'inning_score_diff':'mean'}).reset_index().copy()
re24.rename(columns={'inning_score_diff':'run_expectancy'}, inplace=True)
re24.head()

,on_base,outs_when_up,run_expectancy
0,0.0,0,0.527082
1,0.0,1,0.281229
2,0.0,2,0.107414
3,1.0,0,0.944339
4,1.0,1,0.558057


### RE24 Long Format

In [18]:
re24_long = re24.copy()
# combine out and onbase into base out state
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==0), 'baseOutState'] = '0_0'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==0), 'baseOutState'] = '1_0'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==0), 'baseOutState'] = '2_0'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==0), 'baseOutState'] = '3_0'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==0), 'baseOutState'] = '12_0'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==0), 'baseOutState'] = '13_0'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==0), 'baseOutState'] = '23_0'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==0), 'baseOutState'] = '123_0'
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==1), 'baseOutState'] = '0_1'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==1), 'baseOutState'] = '1_1'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==1), 'baseOutState'] = '2_1'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==1), 'baseOutState'] = '3_1'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==1), 'baseOutState'] = '12_1'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==1), 'baseOutState'] = '13_1'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==1), 'baseOutState'] = '23_1'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==1), 'baseOutState'] = '123_1'
re24_long.loc[(re24_long['on_base']==0)&(re24_long['outs_when_up']==2), 'baseOutState'] = '0_2'
re24_long.loc[(re24_long['on_base']==1)&(re24_long['outs_when_up']==2), 'baseOutState'] = '1_2'
re24_long.loc[(re24_long['on_base']==2)&(re24_long['outs_when_up']==2), 'baseOutState'] = '2_2'
re24_long.loc[(re24_long['on_base']==3)&(re24_long['outs_when_up']==2), 'baseOutState'] = '3_2'
re24_long.loc[(re24_long['on_base']==12)&(re24_long['outs_when_up']==2), 'baseOutState'] = '12_2'
re24_long.loc[(re24_long['on_base']==13)&(re24_long['outs_when_up']==2), 'baseOutState'] = '13_2'
re24_long.loc[(re24_long['on_base']==23)&(re24_long['outs_when_up']==2), 'baseOutState'] = '23_2'
re24_long.loc[(re24_long['on_base']==123)&(re24_long['outs_when_up']==2), 'baseOutState'] = '123_2'
# drop on_base and outs
re24_long.drop(['on_base', 'outs_when_up'], axis=1, inplace=True)
re24_long.rename(columns={'inning_score_diff':'run_expectancy'}, inplace=True)
re24_long

,run_expectancy,baseOutState
0,0.527082,0_0
1,0.281229,0_1
2,0.107414,0_2
3,0.944339,1_0
4,0.558057,1_1
5,0.235467,1_2
6,1.163008,2_0
7,0.719126,2_1
8,0.334304,2_2
9,1.419170,3_0


### RE24 Wide Format:

In [19]:
re24 = re24.pivot(index='on_base', columns='outs_when_up', values='run_expectancy').reset_index().copy()
re24

outs_when_up,on_base,0,1,2
0,0.0,0.527082,0.281229,0.107414
1,1.0,0.944339,0.558057,0.235467
2,2.0,1.163008,0.719126,0.334304
3,3.0,1.419170,0.984796,0.386690
4,12.0,1.569422,0.968574,0.455921
5,13.0,1.764677,1.214943,0.526350
6,23.0,2.039294,1.448111,0.604796
7,123.0,2.428825,1.674826,0.795972


### Turn RE charts into Dict for faster operation

In [20]:
# re24 long:
re24_long_dict = {}
for i in range(24):
    re24_long_dict[re24_long.iloc[i]['baseOutState']] = re24_long.iloc[i]['run_expectancy']

In [21]:
# re24 wide:
re24[3] = 0 # RE is 0 when 3 outs
re24_dict = {}
for a in range(8):
    outs = {}
    re24_dict[re24.iloc[a]['on_base']] = outs
    for b in range(4):
        outs[b] = re24.iloc[a][b]

# Calculate Run Value
RV = Post PA RE - Pre PA RE + Run Scored During PA

In [22]:
data_pa.head()

,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,final_inning
0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0,9
1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0,9
2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0,9
3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0,9
4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1,9


The original Statcast data don't contain base_out_state info after each PA. Since PAs are chronologically ordered, we can access the post base_out_state by looping through each row and get the starting base_out_state from the next row. However, we need to identify the last out from each inning so it doesn't count the starting state from a new inning.

In [23]:
# label last out
data_pa['is_last_out'] = 1
for index, row in data_pa.iloc[:-1].iterrows():
    if data_pa.loc[index, 'inning_topbot'] == data_pa.loc[index+1, 'inning_topbot']:
        data_pa.at[index, 'is_last_out'] = 0
        # current PA is not last_out when the next row is still the same half inning
    else:
        continue

In [24]:
# get the post outs and post on base info
data_pa['post_outs'] = 3
data_pa['post_1b'] = 0
data_pa['post_2b'] = 0
data_pa['post_3b'] = 0
for index, row in data_pa.iloc[:-1].iterrows(): # loop through each row except the last row
    if data_pa.loc[index, 'is_last_out'] == 0:
        data_pa.at[index, 'post_outs'] = data_pa.loc[index+1, 'outs_when_up']
        data_pa.at[index, 'post_1b'] = data_pa.loc[index+1, 'on_1b']
        data_pa.at[index, 'post_2b'] = data_pa.loc[index+1, 'on_2b']
        data_pa.at[index, 'post_3b'] = data_pa.loc[index+1, 'on_3b']
    else:
        continue
        
data_pa.rename(columns={'on_base':'pre_on_base'}, inplace=True)

In [25]:
# combine pre onbase information into "pre_on_base" column
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==0)&(data_pa['on_3b']==0), 'pre_on_base'] = 0
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==0)&(data_pa['on_3b']==0), 'pre_on_base'] = 1
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==1)&(data_pa['on_3b']==0), 'pre_on_base'] = 2
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==0)&(data_pa['on_3b']==1), 'pre_on_base'] = 3
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==1)&(data_pa['on_3b']==0), 'pre_on_base'] = 12
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==0)&(data_pa['on_3b']==1), 'pre_on_base'] = 13
data_pa.loc[(data_pa['on_1b']==0)&(data_pa['on_2b']==1)&(data_pa['on_3b']==1), 'pre_on_base'] = 23
data_pa.loc[(data_pa['on_1b']==1)&(data_pa['on_2b']==1)&(data_pa['on_3b']==1), 'pre_on_base'] = 123
# combine post onbase information into "post_on_base" column
data_pa.loc[(data_pa['post_1b']==0)&(data_pa['post_2b']==0)&(data_pa['post_3b']==0), 'post_on_base'] = 0
data_pa.loc[(data_pa['post_1b']==1)&(data_pa['post_2b']==0)&(data_pa['post_3b']==0), 'post_on_base'] = 1
data_pa.loc[(data_pa['post_1b']==0)&(data_pa['post_2b']==1)&(data_pa['post_3b']==0), 'post_on_base'] = 2
data_pa.loc[(data_pa['post_1b']==0)&(data_pa['post_2b']==0)&(data_pa['post_3b']==1), 'post_on_base'] = 3
data_pa.loc[(data_pa['post_1b']==1)&(data_pa['post_2b']==1)&(data_pa['post_3b']==0), 'post_on_base'] = 12
data_pa.loc[(data_pa['post_1b']==1)&(data_pa['post_2b']==0)&(data_pa['post_3b']==1), 'post_on_base'] = 13
data_pa.loc[(data_pa['post_1b']==0)&(data_pa['post_2b']==1)&(data_pa['post_3b']==1), 'post_on_base'] = 23
data_pa.loc[(data_pa['post_1b']==1)&(data_pa['post_2b']==1)&(data_pa['post_3b']==1), 'post_on_base'] = 123

In [26]:
data_pa.head()

,game_date,game_pk,at_bat_number,game_year,home_team,away_team,player_name,batter,pitcher,inning_topbot,inning,outs_when_up,home_score,away_score,post_home_score,post_away_score,bat_score,post_bat_score,on_1b,on_2b,on_3b,events,delta_home_win_exp,delta_run_exp,PA_score,inning_score,inning_score_diff,final_inning,is_last_out,post_outs,post_1b,post_2b,post_3b,pre_on_base,post_on_base
0,2019-03-20,566083,1,2019,OAK,SEA,"Fiers, Mike",543829,571666,Top,1,0,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.022,-0.248,0,0,0,9,0,1,0,0,0,0.0,0.0
1,2019-03-20,566083,2,2019,OAK,SEA,"Fiers, Mike",571745,571666,Top,1,1,0,0,0,0,0,0,0.0,0.0,0.0,field_out,0.015,-0.177,0,0,0,9,0,2,0,0,0,0.0,0.0
2,2019-03-20,566083,3,2019,OAK,SEA,"Fiers, Mike",457803,571666,Top,1,2,0,0,0,0,0,0,0.0,0.0,0.0,single,-0.012,0.128,0,0,0,9,0,2,1,0,0,0.0,1.0
3,2019-03-20,566083,4,2019,OAK,SEA,"Fiers, Mike",429665,571666,Top,1,2,0,0,0,0,0,0,1.0,0.0,0.0,strikeout,0.022,-0.312,0,0,0,9,1,3,0,0,0,1.0,0.0
4,2019-03-20,566083,5,2019,OAK,SEA,"Gonzales, Marco",657656,594835,Bot,1,0,0,0,0,0,0,0,0.0,0.0,0.0,strikeout,-0.021,-0.249,0,1,1,9,0,1,0,0,0,0.0,0.0


In [27]:
# add pre_base_out_state label
data_pa.loc[(data_pa['pre_on_base']==0)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '0_0'
data_pa.loc[(data_pa['pre_on_base']==1)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '1_0'
data_pa.loc[(data_pa['pre_on_base']==2)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '2_0'
data_pa.loc[(data_pa['pre_on_base']==3)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '3_0'
data_pa.loc[(data_pa['pre_on_base']==12)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '12_0'
data_pa.loc[(data_pa['pre_on_base']==13)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '13_0'
data_pa.loc[(data_pa['pre_on_base']==23)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '23_0'
data_pa.loc[(data_pa['pre_on_base']==123)&(data_pa['outs_when_up']==0), 'pre_baseOutState'] = '123_0'
data_pa.loc[(data_pa['pre_on_base']==0)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '0_1'
data_pa.loc[(data_pa['pre_on_base']==1)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '1_1'
data_pa.loc[(data_pa['pre_on_base']==2)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '2_1'
data_pa.loc[(data_pa['pre_on_base']==3)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '3_1'
data_pa.loc[(data_pa['pre_on_base']==12)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '12_1'
data_pa.loc[(data_pa['pre_on_base']==13)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '13_1'
data_pa.loc[(data_pa['pre_on_base']==23)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '23_1'
data_pa.loc[(data_pa['pre_on_base']==123)&(data_pa['outs_when_up']==1), 'pre_baseOutState'] = '123_1'
data_pa.loc[(data_pa['pre_on_base']==0)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '0_2'
data_pa.loc[(data_pa['pre_on_base']==1)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '1_2'
data_pa.loc[(data_pa['pre_on_base']==2)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '2_2'
data_pa.loc[(data_pa['pre_on_base']==3)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '3_2'
data_pa.loc[(data_pa['pre_on_base']==12)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '12_2'
data_pa.loc[(data_pa['pre_on_base']==13)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '13_2'
data_pa.loc[(data_pa['pre_on_base']==23)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '23_2'
data_pa.loc[(data_pa['pre_on_base']==123)&(data_pa['outs_when_up']==2), 'pre_baseOutState'] = '123_2'

In [28]:
# calculate Pre and Post RE, and Run Value
for index, row in data_pa.iterrows():
    pre_on_base = row.pre_on_base
    pre_outs = row.outs_when_up
    
    post_on_base = row.post_on_base
    post_outs = row.post_outs
    
    data_pa.at[index, 'pre_RE'] = re24_dict[pre_on_base][pre_outs]
    data_pa.at[index, 'post_RE'] = re24_dict[post_on_base][post_outs]

data_pa['post_RE'] = data_pa['post_RE'] + data_pa['PA_score']

In [29]:
data_pa['run_value'] = data_pa['post_RE'] - data_pa['pre_RE']

In [30]:
data_pa_final = data_pa[['game_date', 'game_pk', 'game_year',
       'home_team', 'away_team', 'player_name', 'batter', 'pitcher',
       'pre_baseOutState', 'events', 'run_value']].copy()

In [31]:
data_pa_final.head()

,game_date,game_pk,game_year,home_team,away_team,player_name,batter,pitcher,pre_baseOutState,events,run_value
0,2019-03-20,566083,2019,OAK,SEA,"Fiers, Mike",543829,571666,0_0,field_out,-0.245853
1,2019-03-20,566083,2019,OAK,SEA,"Fiers, Mike",571745,571666,0_1,field_out,-0.173815
2,2019-03-20,566083,2019,OAK,SEA,"Fiers, Mike",457803,571666,0_2,single,0.128054
3,2019-03-20,566083,2019,OAK,SEA,"Fiers, Mike",429665,571666,1_2,strikeout,-0.235467
4,2019-03-20,566083,2019,OAK,SEA,"Gonzales, Marco",657656,594835,0_0,strikeout,-0.245853


# Export RV data

In [36]:
# data_pa_final.to_csv('../data/data_RV.csv')